In [ ]:
!pip install gradio

In [ ]:
import torch
import gradio as gr
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import tempfile

# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Model configs
step = 4
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"

# Load motion adapter
adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo, ckpt), device=device))

# Load AnimateDiff pipeline
pipe = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
pipe.scheduler = EulerDiscreteScheduler.from_config(
    pipe.scheduler.config,
    timestep_spacing="trailing",
    beta_schedule="linear"
)

# Function to generate animation
def generate_animation(prompt):
    try:
        output = pipe(prompt=prompt, guidance_scale=1.0, num_inference_steps=step)
        with tempfile.NamedTemporaryFile(suffix=".gif", delete=False) as tmp:
            export_to_gif(output.frames[0], tmp.name)
            return tmp.name
    except Exception as e:
        return gr.Textbox.update(value=f"❌ Error: {str(e)}")

# Gradio UI
with gr.Blocks(title="AnimateDiff Lightning - Text to GIF", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    <div style='text-align:center; padding:20px;'>
        <h1 style='font-size:2.5rem; color:#4f46e5;'>🎞️ AnimateDiff Lightning</h1>
        <p style='font-size:1.1rem; color:#444;'>Buat animasi GIF dari teks menggunakan model <b>AnimateDiff Lightning</b> berbasis diffusion!</p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1, min_width=400):
            prompt_input = gr.Textbox(
                label="📝 Prompt",
                placeholder="Contoh: Cherry blossoms falling slowly in a peaceful Japanese garden",
                lines=2
            )
            generate_button = gr.Button("🚀 Buat Animasi", variant="primary", size="lg")

        with gr.Column(scale=1, min_width=320):
            output_gif = gr.Image(
                type="filepath",
                width=320,
                show_label=True
            )

    generate_button.click(fn=generate_animation, inputs=prompt_input, outputs=output_gif)

# Jalankan UI
if __name__ == "__main__":
    demo.launch()
